In [1]:
'''ResNet-18 Image classfication for cifar-10 with PyTorch 
Author 'Sun-qian'.
'''
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [2]:
class ResidualBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(ResidualBlock, self).__init__()
        self.left = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outchannel)
            )
    def forward(self, x):
        out = self.left(x)
        out += self.shortcut(x)
        out = F.relu(out)
        return out
 
class ResNet(nn.Module):
    def __init__(self, ResidualBlock, num_classes=10):
        super(ResNet, self).__init__()
        self.inchannel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        self.layer1 = self.make_layer(ResidualBlock, 64,  2, stride=1)
        self.layer2 = self.make_layer(ResidualBlock, 128, 2, stride=2)
        self.layer3 = self.make_layer(ResidualBlock, 256, 2, stride=2)
        self.layer4 = self.make_layer(ResidualBlock, 512, 2, stride=2)
        self.fc = nn.Linear(512, num_classes)
 
    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)   #strides=[1,1]
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)
 
    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out
 
 
def ResNet18():
 
    return ResNet(ResidualBlock)

In [6]:
# 定义是否使用GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
 
# 超参数设置
EPOCH = 135           # 遍历数据集次数
pre_epoch = 0         # 定义已经遍历数据集的次数
BATCH_SIZE = 128      # 批处理尺寸(batch_size)
LR = 0.1              # 学习率
 
    
    
# 准备数据集并预处理
transform_train = transforms.Compose([
    #先四周填充0，再把图像随机裁剪成32*32
    transforms.RandomCrop(32, padding=4),  
    #图像一半的概率翻转，一半的概率不翻转
    transforms.RandomHorizontalFlip(),  
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)), #R,G,B每层的归一化用到的均值和方差
])
 
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
#训练数据集 
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
#生成一个个batch进行批训练，组成batch的时候顺序打乱取
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)   
 
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)




# Cifar-10的标签
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
 
# 模型定义-ResNet
net = ResNet18().to(device)
 
# 定义损失函数和优化方式
criterion = nn.CrossEntropyLoss()  #损失函数为交叉熵，多用于多分类问题
optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4) #优化方式为mini-batch momentum-SGD，并采用L2正则化（权重衰减）

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [8]:
# 定义损失函数和优化方式
criterion = nn.CrossEntropyLoss()  #损失函数为交叉熵，多用于多分类问题
optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4) 

# 训练
outf='./model/'
best_acc = 85  #2 初始化best test accuracy
print("Start Training, Resnet-18!")  # 定义遍历数据集的次数
with open("acc.txt", "w") as f:
    with open("log.txt", "w")as f2:
        for epoch in range(pre_epoch, EPOCH):
            print('\nEpoch: %d' % (epoch + 1))
            net.train()
            sum_loss = 0.0
            correct = 0.0
            total = 0.0
            for i, data in enumerate(trainloader, 0):
                # 准备数据
                length = len(trainloader)
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()

                # forward + backward
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                # 每训练1个batch打印一次loss和准确率
                sum_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += predicted.eq(labels.data).cpu().sum()
            print('[epoch:%d, iter:%d] Loss: %.03f | Acc: %.3f%% '
                  % (epoch + 1, (i + 1 + epoch * length), sum_loss / (i + 1), 100. * correct / total))
            f2.write('%03d  %05d |Loss: %.03f | Acc: %.3f%% '
                  % (epoch + 1, (i + 1 + epoch * length), sum_loss / (i + 1), 100. * correct / total))
            f2.write('\n')
            f2.flush()

            # 每训练完一个epoch测试一下准确率
            print("Waiting Test!")
            with torch.no_grad():
                correct = 0
                total = 0
                for data in testloader:
                    net.eval()
                    images, labels = data
                    images, labels = images.to(device), labels.to(device)
                    outputs = net(images)
                    # 取得分最高的那个类 (outputs.data的索引号)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum()
                print('测试分类准确率为：%.3f%%' % (100 * correct / total))
                acc = 100. * correct / total
                # 将每次测试结果实时写入acc.txt文件中
                print('Saving model......')
                torch.save(net.state_dict(), '%s/net_%03d.pth' % (outf, epoch + 1))
                f.write("EPOCH=%03d,Accuracy= %.3f%%" % (epoch + 1, acc))
                f.write('\n')
                f.flush()
                # 记录最佳测试分类准确率并写入best_acc.txt文件中
                if acc > best_acc:
                    f3 = open("best_acc.txt", "w")
                    f3.write("EPOCH=%d,best_acc= %.3f%%" % (epoch + 1, acc))
                    f3.close()
                    best_acc = acc
        print("Training Finished, TotalEPOCH=%d" % EPOCH)

Start Training, Resnet-18!

Epoch: 1
[epoch:1, iter:391] Loss: 1.415 | Acc: 48.196% 
Waiting Test!
测试分类准确率为：54.320%
Saving model......

Epoch: 2
[epoch:2, iter:782] Loss: 1.174 | Acc: 58.146% 
Waiting Test!
测试分类准确率为：54.880%
Saving model......

Epoch: 3
[epoch:3, iter:1173] Loss: 0.993 | Acc: 64.890% 
Waiting Test!
测试分类准确率为：68.010%
Saving model......

Epoch: 4
[epoch:4, iter:1564] Loss: 0.850 | Acc: 69.948% 
Waiting Test!
测试分类准确率为：70.400%
Saving model......

Epoch: 5
[epoch:5, iter:1955] Loss: 0.726 | Acc: 74.630% 
Waiting Test!
测试分类准确率为：73.780%
Saving model......

Epoch: 6
[epoch:6, iter:2346] Loss: 0.637 | Acc: 77.826% 
Waiting Test!
测试分类准确率为：74.950%
Saving model......

Epoch: 7
[epoch:7, iter:2737] Loss: 0.591 | Acc: 79.604% 
Waiting Test!
测试分类准确率为：77.770%
Saving model......

Epoch: 8
[epoch:8, iter:3128] Loss: 0.554 | Acc: 81.016% 
Waiting Test!
测试分类准确率为：79.210%
Saving model......

Epoch: 9
[epoch:9, iter:3519] Loss: 0.531 | Acc: 81.758% 
Waiting Test!
测试分类准确率为：78.690%
Saving model.